<img src="https://brand.umich.edu/assets/brand/style-guide/logo-guidelines/U-M_Logo-Horizontal-Hex.png" alt="Drawing" style="width: 300px;" align="left"/><br>
    
## Week 4: Dashboard

For this notebook, you are building a prototype dashboard. The specific user persona we have in mind for you to design around is a non-technical instructor who is relatively new to teaching online and has moderate to low data literacy skills. 

The instructor, as we have mentioned in previous Notebooks, is looking to run a report about 25% of the way through the course in order to identify who he or she should do a formal check-in with.
 
To help you in building your dashboard, this notebook provides a brief introduction to the [jupyter-dash extension](https://github.com/plotly/jupyter-dash) for building a [Plotly Dash](https://github.com/plotly/dash) app within Jupyter environments. You can check out the documentation and some tutorials for Plotly Dash [here](https://dash.plotly.com/).

You can also use other packages like `seaborn`. **The only caveat is that the resulting visualization needs to be interactive.**

Resources:
* [A medium blog - Introducing JupyterDash](https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e)
* [Plotly Dash documentation](https://dash.plotly.com/)
* [A Youtube tutorial on dash](https://www.youtube.com/watch?v=hSPmj7mK6ng)
* [Dash gallery](https://dash-gallery.plotly.host/Portal/)

You can also check out [OU Analyse](https://analyse.kmi.open.ac.uk/) as an example of how the authors of the dataset have developed a dashboard for their institution. You can request a demo of the dashboard by entering your email.

In [ ]:
# !pip install jupyter-dash  ## should already be installed

import pandas as pd
import numpy as np

# Import jupyter dash
from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
from dash import dcc
from dash import html

# Import plotly
import plotly.graph_objs as go
import plotly.express as px

# Set up jupyter proxy
JupyterDash.infer_jupyter_proxy_config()

In [1]:
import pandas as pd
import numpy as np

# Import jupyter dash
from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
from dash import dcc
from dash import html

# Import plotly
import plotly.graph_objs as go
import plotly.express as px

In [2]:
lr = pd.read_csv('/Users/nuremek/Downloads/Files 4/learning_resources.csv')
qt = pd.read_csv('/Users/nuremek/Downloads/Files 4/quizzes_tests.csv')
si = pd.read_csv('/Users/nuremek/Downloads/Files 4/student_info_pred.csv')
df = pd.read_csv('/Users/nuremek//Downloads/Files 4/country_indicators.csv')

# A quick tutorial

In essence, a plotly-dash dashboard consists of 3 components:
* The **dash components** (e.g., dropdown, slider, checklist, etc.). See the documentation [here](https://dash.plotly.com/dash-core-components)
* The **plotly** graphs (e.g., linegraph, scatter plot, heatmap, etc.). See the documentation [here](https://plotly.com/python/)
* The **callback** to connects the dash components to plotly graphs, making it an interactive dashboard. See the documentation [here](https://dash.plotly.com/basic-callbacks) 



## Step 1. The Dash components (i.e. layouts)

In [ ]:
score = qt.groupby('assignment_name')['score'].agg({np.mean,np.median, np.std}).reset_index(drop = False)

In [ ]:
app = JupyterDash(__name__)

# Create server variable
server = app.server

# Create a unique list of code_module
assess_list = score['assignment_name'].unique()

# Step 1
app.layout = html.Div([
    
            # Create a html title for the dashboard
            html.H1("This is the title"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id = 'graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id = 'crossfilter-xaxis-column',
                options = [{'label': i, 'value': i} for i in assess_list],
                value ='Quiz 1' # the default assignment
            )])

# Run the app
app.run_server(mode = "inline", port = 8100)

# You will see we have the dropdown menu but nothing happens yet

## Step 2 & 3. Callback and create plotly graph

In [ ]:
app = JupyterDash(__name__)
# Create server variable
server = app.server

In [ ]:
# Create a unique list of code_module
assess_list = score['assignment_name'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a bar chart"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in assess_list],
                value='Quiz 1' # the default
            )])

# Step 2
# Callback using input from dropdown menu to generate graph
# You can have multiple inputs and multiple outputs
@app.callback(
    dash.dependencies.Output('graph-with-dropdown', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(assignment_name):
    filtered_score = score[(score.assignment_name == assignment_name)]
    figure = px.bar(filtered_score, x = 'assignment_name', y = 'mean')
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode = "inline", port = 8101)

In [ ]:
app = JupyterDash(__name__)
# Create server variable
server = app.server

In [ ]:
# Create a unique list of assignment_name
assess_list = score['assignment_name'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a scatter plot"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph'),
    
            # Create a checklist based on assignment_name
            dcc.Checklist(
                    id = 'checklist',
                    options=[{'label': i, 'value': i} for i in assess_list],
                    value=assess_list # Default values contain all assignment_name
            )
])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('checklist', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(values):
    figure = px.scatter(
                     score[score["assignment_name"].isin(values)],
                     x="mean",
                     y="std",
                     color="assignment_name",
                     hover_name="assignment_name")
    return figure # You must return all the output(s) in step 
    
# Run the app
app.run_server(mode="inline", port = 8102)

# Building Your Dashboard (40 points)

Your final product will need to have the following capabilities:

1. (10 points) View multiple students' predicted probabilities of failing the course
2. (10 points) View a student's quiz/test performances alongside a meaningful reference like a course average for a given quiz/test
3. (10 points) View a student's learning resource use alongside a meaningful reference like a course average for a given resource
4. (10 points) Intergrate the above three capabilties into a single, functional dashboard

This notebook is broken out into four sections, three to build individual components and one to integrate them. The final Integrated Dashboard cell will be graded and must include all three specified capabilities. If it does not, points will be awarded based on progress made in each Component cell.

There is a new column in `student_info.csv`, `fail_pred,` that represents the predicted probability of failure for each student from a baseline model.

**Note:** All quizzes, tests, and resources should only include information from on or before day 60 in the course.

### Dictionary
- **student_info.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **gender** = character; M = "male", F = "female"
    - **highest_education** = character; “Some Graduate”, “Some Higher Education”, “High School + Advanced Placement”, “High School”, “No Formal Quals” (Categories ordered from highest documented education level attained to lowest documented education level attained)
    - **disability** = character; Y = "yes", N = "no"
    - **final_result** = character; "Fail", "Pass"
    - **fail_pred** = numeric; predicted probability from sample model


- **quizzes_tests.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **assignment_name** = character; name of graded assignment (Quiz 1-7, Test 1-6, Final Exam)
    - **due_date** = numeric; date assignment was due (indexed as count in days from start of course, i.e., day 0)
    - **weight** = numeric; weight multiplied by score when generating final grade (weight * score / 100)
    - **date_submitted** = numeric; date student submitted assigned (indexed as count in days from start of course, i.e., day 0, NaN means students did not submit assignment)
    - **score** = numeric; score student earned on assignment (0 means students did not submit assignment)


- **learning_resources.csv**
    - **id_student** = numeric; unique identifier for each student in the course
    - **activity_type** = character; overarching label for learning activity students can access (“course_homepage”, “course_page”, “forum”, ‘resource”, “wiki”)
    - **activity_id** = numeric; unique identifier for specific learning activity student accessed within overacting `activity_type`
    - **date** = numeric; date student accessed specific `acitivity_id`  (indexed as count in days from start of course, i.e., day 0)
    - **sum_click** = numeric; count of clicks for `activity_id` on date

## Component 1
1. (10 points) View multiple students' predicted probabilities of failing the course

In [ ]:
app = dash.Dash('prediction')
# Step 1

fig_dropdown = html.Div([
    html.H2(children='Chances of failing by student', style={'text-align': 'center'}),
    dcc.Dropdown(
        id='fig_dropdown',
        options=[{'label': x, 'value': x} for x in si['id_student'].astype(str).unique()],
        value=si['id_student'].astype(str).unique()[0:3],
        style={"width": "60%"},
        multi=True
    )])

fig_plot = html.Div(dcc.Graph(id='fig_plot'))
app.layout = html.Div([fig_dropdown, fig_plot])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('fig_plot', 'figure'),
    [dash.dependencies.Input('fig_dropdown', 'value')])

# Step 2
def update_graph(stu):
    filtered_df = si[(si['id_student'].astype(str).isin(stu))]
    fig = px.bar(filtered_df, x=filtered_df['id_student'].astype(str), y=round(filtered_df['pred_fail'],2), labels={'y': 'Chance of Failing Course', 'x':'Student ID'})
    return fig

# Run the app
app.run_server(mode='inline', debug=True)

## Component 2

2. (10 points) View a student's quiz/test performances alongside a meaningful reference like a course average for a given quiz/test

**Note:** All quizzes, tests, and resources should be from on or before day 60 in the course.

In [ ]:
# merge qt and lr datasets
qt_lr = pd.merge(qt,lr,on='id_student',how='inner')
qt_lr_60 = qt_lr[qt_lr['date_submitted'] <= 60]
avg_score = qt_lr_60['score'].mean()

app = dash.Dash('tests')
# Step 1

stu_dropdown = html.Div([
    html.H2(children='Student quiz/tests scores first 60 days of course', style={'text-align': 'center'}),
    dcc.Dropdown(
        id='stu_dropdown',
        options=[{'label': x, 'value': x} for x in qt_lr_60['id_student'].unique()],
        value=qt_lr_60['id_student'].unique()[0:1],
        style={"width": "60%"},
        multi=True,
        placeholder="Select a student"
    )])

test_dropdown = html.Div([
    dcc.Dropdown(
        id='test_dropdown',
        options=[{'label': x, 'value': x} for x in qt_lr_60['assignment_name'].unique()],
        value=qt_lr_60['assignment_name'].unique()[0:2],
        style={"width": "60%"},
        multi=True,
        placeholder="Select a quiz/test"
    )])

stu_plot = html.Div(dcc.Graph(id='stu_plot'))
app.layout = html.Div([html.Div([stu_dropdown, test_dropdown]), stu_plot])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('stu_plot', 'figure'),
    [dash.dependencies.Input('stu_dropdown', 'value'),
     dash.dependencies.Input('test_dropdown', 'value')])

# Step 2
def update_graph(stu, test):

    filtered_df = qt_lr_60[(qt_lr_60['id_student'].isin(stu)) & (qt_lr_60['assignment_name'].isin(test))]
    score_avg = filtered_df.groupby(['assignment_name']).agg({'score':'mean'}).reset_index()
    fig = px.bar(score_avg, x='assignment_name', y='score', labels={'assignment_name': 'Assignment Name', 'score':'Score'})
    fig.add_hline(y=avg_score, line_width=2, line_dash="dash", line_color="magenta", annotation_text='Score AVG = ' + str(round(avg_score,2)))
    return fig

# Run the app
app.run_server(mode='inline', debug=True)

#raise NotImplementedError()

## Component 3

3. (10 points) View a student's learning resource use alongside a meaningful reference like a course average for a given resource

**Note:** All quizzes, tests, and resources should be from on or before day 60 in the course.

In [ ]:
# merge qt and lr datasets
qt_lr = pd.merge(qt,lr,on='id_student',how='inner')
qt_lr_60 = qt_lr[qt_lr['date_submitted'] <= 60]
avg_score = qt_lr_60['score'].mean()
avg_clicks = qt_lr_60['sum_click'].mean()

app = dash.Dash('resources')
# Step 1

fig_dropdown = html.Div([
    html.H2(children='Learning resources used during the first 60 days of course', style={'text-align': 'center'}),
    dcc.Dropdown(
        id='fig_dropdown',
        options=[{'label': x, 'value': x} for x in ['Scores', 'Clicks']],
        value='Scores',
        style={"width": "60%"},
        placeholder="Select a chart"
    )])

stu_dropdown = html.Div([
     dcc.Dropdown(
        id='stu_dropdown',
        options=[{'label': x, 'value': x} for x in qt_lr_60['id_student'].unique()],
        value=qt_lr_60['id_student'].unique()[0:1],
        style={"width": "60%"},
        multi=True,
        placeholder="Select a student"
    )])

activity_dropdown = html.Div([
    dcc.Dropdown(
        id='activity_dropdown',
        options=[{'label': x, 'value': x} for x in qt_lr_60['activity_type'].unique()],
        value=qt_lr_60['activity_type'].unique()[0:2],
        style={"width": "60%"},
        multi=True,
        placeholder="Select a resource"
    )])

stu_plot = html.Div(dcc.Graph(id='stu_plot'))

app.layout = html.Div([html.Div([fig_dropdown, stu_dropdown, activity_dropdown]), stu_plot])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('stu_plot', 'figure'),
    [dash.dependencies.Input('fig_dropdown', 'value'),
     dash.dependencies.Input('stu_dropdown', 'value'),
     dash.dependencies.Input('activity_dropdown', 'value')])

# Step 2
def update_graph(figs, stu, act):

    if figs == 'Scores':
        
        filtered_df = qt_lr_60[(qt_lr_60['id_student'].isin(stu)) & (qt_lr_60['activity_type'].isin(act))]
        score_avg = filtered_df.groupby(['activity_type']).agg({'score':'mean'}).reset_index()
        fig1 = px.bar(score_avg, x='activity_type', y=round(score_avg['score'],2), labels={'activity_type': 'Learning Resource', 'y':'Score'})
        fig1.add_hline(y=avg_score, line_width=2, line_dash="dash", line_color="magenta", annotation_text='Score AVG = ' + str(round(avg_score,2)))
        return fig1

    elif figs == 'Clicks':
        
        filtered_df = qt_lr_60[(qt_lr_60['id_student'].isin(stu)) & (qt_lr_60['activity_type'].isin(act))]
        clicks_avg = filtered_df.groupby(['activity_type']).agg({'sum_click':'mean'}).reset_index()
        fig2 = px.bar(clicks_avg, x='activity_type', y=round(clicks_avg['sum_click'],2), labels={'activity_type': 'Learning Resource', 'y':'Score'})
        fig2.add_hline(y=avg_clicks, line_width=2, line_dash="dash", line_color="magenta", annotation_text='Score AVG = ' + str(round(avg_clicks,2)))
        return fig2
# Run the app
app.run_server(mode='inline', debug=True)

# raise NotImplementedError()

## [BONUS] Component 4
Includes total students by gender, education, and disability.

In [ ]:
# bin pred_fail into buckets and group by count ot id_student to get total students by bin
bins_si = si
bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
labels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
bins_si['bin'] = pd.cut(si['pred_fail'], bins)
bins_si['bin_label'] = pd.cut(bins_si['pred_fail'], bins=bins, labels=labels)
bins_agg = bins_si.groupby(['bin_label']).agg({'id_student':'count'}).reset_index(drop = False)
bins_gen = bins_si.groupby(['gender']).agg({'id_student':'count'}).reset_index(drop = False)
bins_edu = bins_si.groupby(['highest_education']).agg({'id_student':'count'}).reset_index(drop = False)
bins_dis = bins_si.groupby(['disability']).agg({'id_student':'count'}).reset_index(drop = False)

# overall view for count of students by attributes fail prediction, gender, education, and disability

figs = ['Fail Prediction', 'Gender', 'Education', 'Disability']
app = dash.Dash()
# Step 1

fig_dropdown = html.Div([
    html.H2(children='Student Information Bar Chart', style={'text-align': 'center'}),
    dcc.Dropdown(
        id='fig_dropdown',
        options=[{'label': x, 'value': x} for x in figs],
        value=figs[0],
        style={"width": "60%"}
    )])

fig_plot = html.Div(dcc.Graph(id='fig_plot'))
app.layout = html.Div([fig_dropdown, fig_plot])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('fig_plot', 'figure'),
    [dash.dependencies.Input('fig_dropdown', 'value')])

# Step 2
def update_graph(figs):
    if figs == 'Fail Prediction':
        fig1 = px.bar(bins_agg, x=bins_agg['bin_label'], y=bins_agg['id_student'], labels={'bin_label': 'Fail Prediction', 'id_student':'Total Unique Students'})
        return fig1
    if figs == 'Gender':
        fig2 = px.bar(bins_gen, x=bins_gen['gender'], y=bins_gen['id_student'], color=bins_gen['gender'], labels={'gender': 'Gender', 'id_student':'Total Unique Students'})
        return fig2
    if figs == 'Education':
        fig3 = px.bar(bins_edu, x=bins_edu['highest_education'], y=bins_edu['id_student'], color=bins_edu['highest_education'], labels={'education': 'Highest Education', 'id_student':'Total Unique Students'})
        return fig3
    if figs == 'Disability':
        fig4 = px.bar(bins_dis, x=bins_dis['disability'], y=bins_dis['id_student'], color=bins_dis['disability'], labels={'disability': 'Disability', 'id_student':'Total Unique Students'})
        return fig4

# Run the app
app.run_server(mode='inline', debug=True)

---

## Integrated Dashboard

4. (10 points) Intergrate the above three capabilties into a single, functional dashboard

In [4]:
# merge qt and lr datasets
qt_lr = pd.merge(qt,lr,on='id_student',how='inner')
qt_lr_60 = qt_lr[qt_lr['date_submitted'] <= 60]
avg_score = qt_lr_60['score'].mean()
avg_clicks = qt_lr_60['sum_click'].mean()
avg_pred = si['pred_fail'].mean()

# for the fourth chart, create bin pred_fail into buckets and group by count ot id_student to get total students by bin
bins_si = si
bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
labels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
bins_si['bin'] = pd.cut(si['pred_fail'], bins)
bins_si['bin_label'] = pd.cut(bins_si['pred_fail'], bins=bins, labels=labels)
bins_agg = bins_si.groupby(['bin_label']).agg({'id_student':'count'}).reset_index(drop = False)
bins_gen = bins_si.groupby(['gender']).agg({'id_student':'count'}).reset_index(drop = False)
bins_edu = bins_si.groupby(['highest_education']).agg({'id_student':'count'}).reset_index(drop = False)
bins_dis = bins_si.groupby(['disability']).agg({'id_student':'count'}).reset_index(drop = False)

#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__)#external_stylesheets = external_stylesheets)

# create multi dropdorn chart for failure prediction by student
pred_dropdown = html.Div([
    html.H2(children='Chances of failing by student', 
            style={'color': '#00274C',
                   #'backgroundColor': '#00274C',
                   'text-align': 'center'}),
    dcc.Dropdown(
        id='pred_dropdown',
        options=[{'label': x, 'value': x} for x in si['id_student'].astype(str).unique()],
        value=si['id_student'].astype(str).unique()[0:3],
        style={"width": "50%"},
        multi=True),
    html.Div(dcc.Graph(id='pred_plot'))], 
    className = 'six columns')

# create multi dropdown chart for quiz/test results by student
stu_dropdown_qt = html.Div([
    html.H2(children='Student quiz/tests scores first 60 days of course', 
            style = {'color': '#00274C',
                     #'backgroundColor': '#00274C',
                     'text-align': 'center'}),
    dcc.Dropdown(
        id='stu_dropdown_qt',
        options=[{'label': x, 'value': x} for x in qt_lr_60['id_student'].unique()],
        value=qt_lr_60['id_student'].unique()[0:3],
        style={"width": "50%"},
        multi=True,
        placeholder="Select a student"
    )
], className = 'six columns')

test_dropdown = html.Div([
    dcc.Dropdown(
        id='test_dropdown',
        options=[{'label': x, 'value': x} for x in qt_lr_60['assignment_name'].unique()],
        value=qt_lr_60['assignment_name'].unique()[0:4],
        style={"width": "50%"},
        multi=True,
        placeholder="Select a quiz/test"),
    html.Div(dcc.Graph(id='test_plot'))], 
    className = 'six columns')

# create learning resource multi dropdown by scores and clicks and by student
stu_dropdown_lr = html.Div([
    html.H2(children='Learning resources used during the first 60 days of course by score and resource clicks', 
            style={'color': '#00274C',
                   #'backgroundColor': '#00274C',
                   'text-align': 'center'}),
    dcc.Dropdown(
        id='stu_dropdown_lr',
        options=[{'label': x, 'value': x} for x in qt_lr_60['id_student'].unique()],
        value=qt_lr_60['id_student'].unique()[0:3],
        style={"width": "50%"},
        multi=True,
        placeholder="Select a student"
    )
], className = 'six columns')

activity_dropdown = html.Div([
    dcc.Dropdown(
        id='activity_dropdown',
        options=[{'label': x, 'value': x} for x in qt_lr_60['activity_type'].unique()],
        value=qt_lr_60['activity_type'].unique()[0:5],
        style={"width": "50%"},
        multi=True,
        placeholder="Select a resource"),
    html.Div(dcc.Graph(id='activity_score_plot')),
    html.Div(dcc.Graph(id='activity_click_plot'))], 
    className='six columns')

# create student demographics charts
demo_dropdown = html.Div([
    html.H2(children='Student Demographics', style={'text-align': 'center'}),
    html.Div(dcc.Graph(id='demo_score_plot')),
    html.Div(dcc.Graph(id='demo_gender_plot')),
    html.Div(dcc.Graph(id='demo_education_plot')),
    html.Div(dcc.Graph(id='demo_disability_plot'))],
        className='six columns')


# Step 2
# put all the charts together into a single dashboard
app.layout = html.Div(children=[
                 html.Div([
                 html.H1(children = "Student Dashboard",
                         style = {'color': '#FFCB05',
                                  'backgroundColor': "#00274C",
                                  'text-align':'center'})], className = 'row'),
                 # create two rows in dashboard, each it's own html.Div
                 html.Div([pred_dropdown, stu_dropdown_qt, test_dropdown], className='row'),
                 html.Div([stu_dropdown_lr, activity_dropdown], className='row'),
                 html.Div([demo_dropdown], className='row')
])


# Callback using inputs from the checklist to generate the graph
@app.callback([
    dash.dependencies.Output('pred_plot', 'figure'),
    dash.dependencies.Output('test_plot', 'figure'),
    dash.dependencies.Output('activity_score_plot', 'figure'),
    dash.dependencies.Output('activity_click_plot', 'figure'),
    dash.dependencies.Output('demo_score_plot', 'figure'),
    dash.dependencies.Output('demo_gender_plot', 'figure'),
    dash.dependencies.Output('demo_education_plot', 'figure'),
    dash.dependencies.Output('demo_disability_plot', 'figure')
],
    [dash.dependencies.Input('pred_dropdown', 'value'),
     dash.dependencies.Input('stu_dropdown_qt', 'value'),
     dash.dependencies.Input('test_dropdown', 'value'),
     dash.dependencies.Input('stu_dropdown_lr', 'value'),
     dash.dependencies.Input('activity_dropdown', 'value'),
    ])


# Step 2
def update_graph(pred, stu_qt, test, stu_lr, act):
    # return fail prediction graph
    filtered_df = si[(si['id_student'].astype(str).isin(pred))]
    fig1 = px.bar(filtered_df, x=filtered_df['id_student'].astype(str), y=round(filtered_df['pred_fail'],2), labels={'y': 'Chance of Failing Course', 'x':'Student ID'})
    
    # return quiz/test graph
    filtered_df = qt_lr_60[(qt_lr_60['id_student'].isin(stu_qt)) & (qt_lr_60['assignment_name'].isin(test))]
    score_avg = filtered_df.groupby(['assignment_name']).agg({'score':'mean'}).reset_index()
    fig2 = px.bar(score_avg, x='assignment_name', y='score', labels={'assignment_name': 'Assignment Name', 'score':'Score'})
    fig2.add_hline(y=avg_score, line_width=2, line_dash="dash", line_color="magenta", annotation_text='Score AVG = ' + str(round(avg_score,2)))

    # return resource clicks and scores graph
    #if figs_chart3 == 'Scores':
        
    filtered_df = qt_lr_60[(qt_lr_60['id_student'].isin(stu_lr)) & (qt_lr_60['activity_type'].isin(act))]
    score_avg = filtered_df.groupby(['activity_type']).agg({'score':'mean'}).reset_index()
    fig3 = px.bar(score_avg, x='activity_type', y=round(score_avg['score'],2), labels={'activity_type': 'Learning Resource', 'y':'Score'})
    fig3.add_hline(y=avg_score, line_width=2, line_dash="dash", line_color="magenta", annotation_text='Score AVG = ' + str(round(avg_score,2)))

    filtered_df = qt_lr_60[(qt_lr_60['id_student'].isin(stu_lr)) & (qt_lr_60['activity_type'].isin(act))]
    clicks_avg = filtered_df.groupby(['activity_type']).agg({'sum_click':'mean'}).reset_index()
    fig4 = px.bar(clicks_avg, x='activity_type', y=round(clicks_avg['sum_click'],2), labels={'activity_type': 'Learning Resource', 'y':'Clicks'})
    fig4.add_hline(y=avg_clicks, line_width=2, line_dash="dash", line_color="magenta", annotation_text='Clicks AVG = ' + str(round(avg_clicks,2)))

    fig5 = px.bar(bins_agg, x=bins_agg['bin_label'], y=bins_agg['id_student'], labels={'bin_label': 'Fail Prediction', 'id_student':'Total Unique Students'})
    fig5.add_vline(x=avg_pred, line_width=2, line_dash="dash", line_color="magenta", annotation_text='Prediction AVG = ' + str(round(avg_pred,2)))

    fig6 = px.bar(bins_gen, x=bins_gen['gender'], y=bins_gen['id_student'], color=bins_gen['gender'], labels={'gender': 'Gender', 'id_student':'Total Unique Students'})
    fig7 = px.bar(bins_edu, x=bins_edu['highest_education'], y=bins_edu['id_student'], color=bins_edu['highest_education'], labels={'highest_education': 'Highest Education', 'id_student':'Total Unique Students'})
    fig8 = px.bar(bins_dis, x=bins_dis['disability'], y=bins_dis['id_student'], color=bins_dis['disability'], labels={'disability': 'Disability', 'id_student':'Total Unique Students'})
    
    return fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8

# Run the app
if __name__ == '__main__':
    app.run_server(mode='inline', debug=True)
# raise NotImplementedError()

## 2. Interpret (10 points)

After developing your integrated dashboard, provide your thoughs on the following questions:

2.1 How well will your integrated dashboard help an instructor identify which students to check-in with, and why? (5 points)

<font color='#00274C'>

The dashboard includes total eight charts to better understand where a student may need intervention. Below are ways in which each chart can assist answering an instructor's questions for actionable insights:
    
- Course failure prediction by student: 
    
    - Which students are likely to fail a course?
    
- Quiz/test scores by student including overall score average: 
    
    - What is a student's score by quiz/test before 60 days? 
    - What is a student's test score compared to average?
    - If multiple students are selected, what is their average test score by test?
    
- Resources used by student's total and average scores
    
    - What is a student's average score by learning resource used? What about multiple students's scores?
    - Which learning resource was most effective in the student receiving higher average score?
    
- Resources used by student's total and average clicks
    
    - Which learning resource was clicked on the most by a student?  By multiple students?
    - Compared to average clicks by learning resource, which resource did the student prefer to use more often?
    
- Total students by fail predictions
    
    - What is the breakout of students likely to fail compared to average predicted score?
    
- Total students by gender
    
    - How many students are in the course by gender?
    
- Total students by education
    
    - How many students have at most high school education? Post high school level?
    
- Total students by disability
    
    - How many students are disabled vs non disabled in the course?
    
</font>

2.2 Based on the readings and videos, what would you recommend as improvements to your dashboard, and why? (5 points)

<font color='#00274C'>

The article <a href="https://link-springer-com.proxy.lib.umich.edu/article/10.1007/s11121-017-0842-9" style="color:purple;">Using Technology and Assessment to Personalize Instruction: Preventing Reading Problems</a> states that 30% in their sample were not able to read proficiently in 4th grade, and the percentage was higher for those living in poverty. An effective method used is the personalized A2i student to instructor intervention, which works as long as it's sustained for a couple years (first to third grade). Another intervention is for improving reading comprehension, WKe-Book, which is mostly self-serve to students, guided by tutors with instructor input removed. Wke-Book seems to be most effective when followed over a couple years (third to fifth grade) as well. It would be great to include data with interventions like A2i and WKE-Book in the dashboard to identify success, i.e. how many years the intervention should take place, pass rate at the student level, etc.

In addition, demographics including disability, socioeconomic status, and neighborhood are all factors that contribute to a student's learning and development. When creating a dashboard that doesn't include such data, the dashboard could result in omission bias and the recommendations will favor unfairly to certain groups of students. As mentioned in <a href=“https://link-springer-com.proxy.lib.umich.edu/article/10.1007/s11121-017-0842-9” style="color:purple;">Using Technology and Assessment to Personalize Instruction: Preventing Reading Problems</a>, school systems are not widely using "personalized instruction," which makes it difficult to interpret whether or not certain interventions are working. And if they are, can they be applied to different schools and locations? Different demographic of students? Possibly more data mining for datasets from various schools globally (not including PII) to better understand which features to include in the dashboard.
</font>